In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from kiwipiepy import Kiwi

from collections import Counter

import time

In [2]:
kiwi = Kiwi()

def count_word(text, word_counts):
    # kiwi.analyze()로 분석 후, 품사가 'NNG'(일반 명사), 'NNP'(고유 명사)인 경우만 추출
    nouns = [token.form for token in kiwi.analyze(text)[0][0] if token.tag.startswith('NN')]
    new_word_counts = Counter(nouns) + word_counts
    return new_word_counts

In [7]:
driver = webdriver.Chrome(service=Service("./driver/chromedriver"))

# url에 포함되어 있는 parameters
# 정렬 : sort={"0":관련도순, "1":최신순, "2":오래된순}
# 기간 : (시작기간)ds=YYYY.MM.DD, (끝기간)de=YYYY.MM.DD
# 유형 : photo={"0":전체, "1":포토, "2":동영상, "3":지면기사, ...}
# mynews=1, office_type=3, office_category=3으로 수정
company_name = "폭락"  # 검색어
date = "2024.08.05"      # 검색기간(하루로 제한 -> 추후 수정 예정)
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&query={company_name}&sm=tab_opt&sort=0&photo=3&field=0&pd=3&ds={date}&de={date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date.replace(".","")}to{date.replace(".","")}&is_sug_officeid=0&office_category=3&service_area=0"
driver.get(url)
time.sleep(2)            # 사이트 차단 방지를 위한 멈춤

In [8]:
# 기간 내 기사들의 링크 모두 가져오기

# 모든 기사가 보일때까지 스크롤
len_before_scroll = 0
while (True):
    # 현재 로드된 뉴스 기사 목록을 모두 가져온다 (CSS 선택자는 실제 페이지에 맞게 수정)
    articles = driver.find_elements(By.CSS_SELECTOR, "div.BHQHyn3Flk5rFBSacJkG")

    # 스크롤 후 기사 수
    len_after_scroll = len(articles)

    if len(articles)>50:
        break

    # 목록의 끝에서 두 번째 항목을 타겟으로 지정 (마지막 항목은 푸터와 겹칠 수 있음)
    if len(articles) > 1:
        target_element = articles[-2]
        
        # JavaScript를 실행하여 해당 요소가 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView()", target_element)

        time.sleep(1)

    # 스크롤 전후 기사 수 비교 (같다면 더이상 스크롤 할 필요 X)
    if len_before_scroll == len_after_scroll:
        break

    len_before_scroll = len_after_scroll

In [ ]:
# 모든 기사들의 데이터 가져오기

news = driver.find_elements(By.CSS_SELECTOR, "div.BHQHyn3Flk5rFBSacJkG")
news_links =[]
word_counts=Counter()
for each in news:
    link_btn = each.find_element(By.CLASS_NAME, "GUWgsNcVrWa67MoYor6N.xR1x3GgF_MIcYPUZNqEu")
    link_btn.click()
    time.sleep(1)

    # 새롭게 생긴 탭으로 핸들 넘기기
    original_window_handle = driver.current_window_handle
    new_tab_handle = None
    for handle in driver.window_handles:
        if handle != original_window_handle:
            new_tab_handle = handle
            break
    driver.switch_to.window(new_tab_handle)

    # 제목 가져오기
    title = driver.find_element(By.CLASS_NAME, "media_end_head_title")
    # print(title.text)
    # print("---------------------------------------------------------")

    # 기사 내용 가져오기
    article = driver.find_element(By.TAG_NAME, "article")
    word_counts = count_word(article.text, word_counts)
    # print(article.text)
    # print("=======================================================")

    # 다시 탭 닫고 핸들 넘기기
    driver.close()
    driver.switch_to.window(original_window_handle)
    time.sleep(1)

In [ ]:
print(word_counts)

In [ ]:
driver.quit()